
# Havayolu Rötar Analizi (Airlines Delay Analysis)

Bu çalışma, GitHub üzerinde bulunan **airlens.csv** veri seti kullanılarak uçuş rötarlarını analiz eder. 
Hedefler:
- Rötara etki eden faktörleri belirlemek (hava yolu, hat, şehir/ülke, saat, gün vb.)
- Zaman serisi trendlerini ve mevsimselliği incelemek
- Basit sınıflandırma modeli ile **rötar olur/olmaz** tahmini yapmak

**English:** This project analyzes flight delays using the **airlens.csv** dataset. We explore drivers of delays, time series patterns, and train a simple classifier to predict whether a flight will be delayed.



## Kurulum ve Çalıştırma

- Python 3.10+ önerilir. Aşağıdaki paketler gereklidir (requirements.txt dosyasında da var):
  - pandas, numpy, matplotlib, seaborn, scikit-learn, statsmodels, pyarrow (opsiyonel, hızlı okuma için)
- Dosya: `airlens.csv` bu notebook ile aynı klasörde veya `data/` altında bulunabilir.

> **Not:** Veri şemanız farklıysa (sütun adları), aşağıdaki **Akıllı Sütun Eşleştirme** bölümünde anahtar sütunlar otomatik bulunmaya çalışılır.


In [ ]:

# Gerekli kütüphaneler
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from statsmodels.tsa.seasonal import seasonal_decompose
from datetime import datetime

pd.set_option('display.max_columns', 120)
pd.set_option('display.width', 140)


In [ ]:

# Dosya konumu (gerekirse burayı düzenleyin)
DATA_PATH_CANDIDATES = [
    'airlens.csv',
    'data/airlens.csv',
    'airlines.csv',
    'data/airlines.csv'
]

DATA_PATH = None
for p in DATA_PATH_CANDIDATES:
    if os.path.exists(p):
        DATA_PATH = p
        break

if DATA_PATH is None:
    print("UYARI: Veri dosyası bulunamadı. Lütfen 'airlens.csv' dosyasını bu klasöre veya 'data/' klasörüne koyun ve yeniden çalıştırın.")
else:
    print(f"Veri dosyası bulundu: {DATA_PATH}")


In [ ]:

# Veriyi yükleme
if DATA_PATH is not None:
    try:
        df = pd.read_csv(DATA_PATH, low_memory=False)
    except Exception as e:
        print("Veri okuma sırasında hata:", e)
        raise e

    print("Boyut:", df.shape)
    display(df.head())
    display(df.tail())
    display(df.sample(min(5, len(df))))

    print("\n--- INFO ---")
    print(df.info())

    print("\nEksik değer sayıları:")
    display(df.isna().sum().sort_values(ascending=False).head(20))



## Akıllı Sütun Eşleştirme (Smart Column Mapping)
Aşağıdaki hücre, veri setindeki muhtemel sütunları (tarih, kalkış/varış gecikmesi, havayolu, kalkış-varış havaalanı vb.) otomatik olarak bulmaya çalışır.


In [ ]:

# Anahtar sütun isimlerini olası varyasyonlara göre bulma
def find_col(candidates):
    for c in candidates:
        for col in df.columns:
            if col.strip().lower() == c:
                return col
    for c in candidates:
        for col in df.columns:
            if c in col.strip().lower():
                return col
    return None

col_map = {}
col_map['date'] = find_col(['date', 'fl_date', 'flight_date', 'scheduled_departure', 'time', 'timestamp'])
col_map['carrier'] = find_col(['airline', 'carrier', 'unique_carrier', 'airline_name'])
col_map['origin'] = find_col(['origin', 'origin_airport', 'origin_city', 'from'])
col_map['dest'] = find_col(['dest', 'destination', 'dest_airport', 'to'])
col_map['dep_delay'] = find_col(['dep_delay', 'departure_delay', 'depart_delay'])
col_map['arr_delay'] = find_col(['arr_delay', 'arrival_delay', 'arrive_delay'])
col_map['cancelled'] = find_col(['cancelled', 'canceled', 'is_cancelled'])
col_map['diverted'] = find_col(['diverted', 'is_diverted'])
col_map['country'] = find_col(['country'])
col_map['city'] = find_col(['city'])

col_map


In [ ]:

# Tarih sütununu parse et ve temel zaman özelliklerini çıkar
if 'df' in globals() and len(df) > 0 and col_map['date'] is not None:
    df[col_map['date']] = pd.to_datetime(df[col_map['date']], errors='coerce')
    df['year'] = df[col_map['date']].dt.year
    df['month'] = df[col_map['date']].dt.month
    df['day'] = df[col_map['date']].dt.day
    df['dayofweek'] = df[col_map['date']].dt.dayofweek
    df['hour'] = df[col_map['date']].dt.hour
else:
    print("Not: Tarih sütunu bulunamadı, zaman bazlı özellikler oluşturulmadı.")



## Hedef Değişken (Target) — `is_delayed`
Eğer `arr_delay` veya `dep_delay` ≥ 15 dk ise 1 (rötarlı), aksi halde 0.


In [ ]:

# is_delayed hedefini oluştur
if 'df' in globals() and len(df) > 0:
    df['dep_delay_filled'] = df[col_map['dep_delay']].fillna(0) if col_map['dep_delay'] else 0
    df['arr_delay_filled'] = df[col_map['arr_delay']].fillna(0) if col_map['arr_delay'] else 0

    if (col_map['dep_delay'] is not None) or (col_map['arr_delay'] is not None):
        delays = []
        for i in range(len(df)):
            dd = df['dep_delay_filled'].iloc[i] if col_map['dep_delay'] else 0
            ad = df['arr_delay_filled'].iloc[i] if col_map['arr_delay'] else 0
            try:
                dd = float(dd)
            except:
                dd = np.nan
            try:
                ad = float(ad)
            except:
                ad = np.nan
            delayed = 1 if ((not np.isnan(dd) and dd >= 15) or (not np.isnan(ad) and ad >= 15)) else 0
            delays.append(delayed)
        df['is_delayed'] = delays
        print("is_delayed sütunu oluşturuldu. Oran: %.2f%%" % (100*df['is_delayed'].mean()))
    else:
        print("Uyarı: dep/arr delay sütunları bulunamadı, 'is_delayed' hedefi oluşturulamadı.")



## Keşifsel Veri Analizi (EDA)
Rötara en çok katkı yapan hava yolu, hat, şehir/ülke gibi kırılımları inceleyelim.


In [ ]:

def plot_rate_by(col_name, top_n=10):
    if col_name not in df.columns:
        print(f"{col_name} sütunu yok.")
        return
    if 'is_delayed' not in df.columns:
        print("is_delayed yok, önce hedefi oluşturun.")
        return
    
    grp = df.groupby(col_name)['is_delayed'].mean().sort_values(ascending=False).head(top_n)
    ax = grp.plot(kind='bar', figsize=(8,4), title=f"Rötar Oranı - {col_name} (Top {top_n})")
    ax.set_ylabel("Oran")
    ax.set_xlabel(col_name)
    plt.tight_layout()
    plt.show()

for c in [col_map['carrier'], col_map['origin'], col_map['dest'], col_map['city'], col_map['country']]:
    if c is not None:
        plot_rate_by(c)



## Zaman Serisi Analizi
Tarihe göre günlük rötar trendi ve mevsimsellik.


In [ ]:

if ('is_delayed' in df.columns) and (col_map['date'] is not None):
    ts = df.set_index(col_map['date']).resample('D')['is_delayed'].sum().dropna()
    if len(ts) > 10:
        result = seasonal_decompose(ts, model='additive', period=7)
        fig = result.plot()
        fig.set_size_inches(10, 7)
        plt.tight_layout()
        plt.show()
    else:
        print("Zaman serisi yeterli uzunlukta değil.")
else:
    print("Tarih veya is_delayed bulunamadı, zaman serisi analizi atlandı.")



## Sınıflandırma ile Rötar Tahmini
Basit bir model (Logistic Regression / RandomForest) ile `is_delayed` tahmini.


In [ ]:

features = []
for key in ['carrier', 'origin', 'dest', 'city', 'country', 'year', 'month', 'dayofweek', 'hour']:
    col = col_map.get(key) if key in ['carrier','origin','dest','city','country'] else key
    if col is not None and col in df.columns:
        features.append(col)

if 'is_delayed' in df.columns and len(features) > 0:
    X = df[features].copy()
    y = df['is_delayed'].astype(int)

    num_cols = [c for c in ['year','month','dayofweek','hour'] if c in X.columns]
    cat_cols = [c for c in X.columns if c not in num_cols]

    pre = ColumnTransformer(
        transformers=[
            ('num', 'passthrough', num_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
        ]
    )

    models = {
        "LogisticRegression": LogisticRegression(max_iter=200),
        "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42)
    }

    for name, model in models.items():
        clf = Pipeline(steps=[('pre', pre), ('model', model)])
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        print(f"\n=== {name} ===")
        print(classification_report(y_test, y_pred, digits=3))

        try:
            y_proba = clf.predict_proba(X_test)[:,1]
            print("ROC-AUC:", roc_auc_score(y_test, y_proba))
        except Exception as e:
            print("ROC-AUC hesaplanamadı:", e)
else:
    print("Modelleme için hedef veya özellikler hazır değil.")



## Sonuçlar ve Öneriler
- Rötar oranı yüksek kırılımlar belirlendi.
- Zaman serisi trendi ve mevsimselliğe dair içgörüler elde edildi.
- Basit modellerle gecikme tahmini yapıldı.

> Sonraki adımlar: Hava durumu, yoğunluk, tatiller gibi ek özelliklerle gelişmiş modeller (XGBoost/LightGBM).


In [ ]:

# (Opsiyonel) Örnek çıktı kaydet
if 'is_delayed' in df.columns:
    df.head(1000).to_csv('processed_sample.csv', index=False)
    print("processed_sample.csv kaydedildi.")
